# Brainstorming Agent for Product Owner

This notebook demonstrates a sophisticated brainstorming agent designed for Product Owners (POs) using LangGraph's ReAct architecture. The agent integrates web browsing, memory management, dynamic PRD generation, and human-in-the-loop interactions to facilitate iterative product development discussions.

## Overview
- **Architecture**: ReAct (Reasoning + Acting) with conditional routing and tool integration.
- **Key Features**: Web research via Tavily API, persistent memory, AI-driven PRD creation, and collaborative refinement.
- **Use Case**: POs brainstorming features (e.g., reporting for Power Cash), with the agent handling research, analysis, and documentation.

This builds on concepts from previous modules (e.g., chains, routers, agents) by adding advanced state management and multi-tool orchestration.

In [6]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt tavily-python

## Section Explanation: Setup and Import

**How it Works**: This section establishes the foundational environment by installing dependencies and importing necessary modules for LangGraph-based agent development. It ensures compatibility with external APIs like Tavily for enhanced web search capabilities.

**What it Does**:
- **Package Installation**: Uses `pip` to install `langchain-openai` (for LLM integration), `langgraph` (for graph orchestration), `langgraph-prebuilt` (for built-in tools and conditions), and `tavily-python` (for web search).
- **Module Imports**: Imports core LangChain components (e.g., `ChatOpenAI` for model interaction, `tool` decorator for function binding), LangGraph primitives (e.g., `StateGraph`, `MessagesState`), and utilities (e.g., `add_messages` reducer for state updates).
- **API Configuration**: Defines a helper function `_set_env` to securely prompt for API keys (OpenAI for LLM, Tavily for search), preventing hardcoding and ensuring runtime flexibility.
- **Model Initialization**: Instantiates `ChatOpenAI` with GPT-4o, providing a high-capability LLM backbone for reasoning and tool calling.

This setup mirrors best practices in LangGraph tutorials, ensuring reproducibility and security.

In [21]:
import os
import getpass
from typing import Dict, List
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages

# Setup API key (use Grok or OpenAI)
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")  # Replace with Grok API if needed

# LLM (use Grok Code Fast if available)
llm = ChatOpenAI(model="gpt-4o")  # Replace with Grok model if integrated

## Section Explanation: Tools

**How it Works**: Defines a suite of Python functions decorated with `@tool`, enabling the LLM to invoke external capabilities dynamically. Each tool is bound to the model via `bind_tools`, allowing the agent to reason about when and how to use them based on user input.

**What it Does**:
- **`browse_web`**: Leverages Tavily API for semantic web search. It queries the web, retrieves top results (up to 3), and formats them as title-content pairs. This enhances research accuracy compared to basic APIs, supporting real-time information gathering for brainstorming.
- **`save_to_memory`**: Appends user-provided information to a global `memory` list. This implements persistent state across graph invocations, allowing the agent to reference prior context (e.g., previous research findings) in subsequent interactions.
- **`read_file`**: Reads the contents of any file and returns it to the user. Useful for examining existing documents, code files, or configuration files during brainstorming sessions.
- **`edit_file`**: Performs precise text replacements in existing files. Takes an old string and new string, finds the exact match, and replaces it. Essential for making targeted edits to documents or code.
- **`apply_patch`**: Applies patches or diffs to files. Supports unified diff format with +/- indicators for additions and deletions. Useful for applying code changes or document updates from external sources.
- **`generate_prd`**: Uses the LLM to dynamically generate or update a comprehensive Product Requirements Document (PRD) in Markdown. Only activates when user explicitly requests PRD generation or editing. If updating existing PRD, it reads the current file and incorporates new information while preserving existing content. The generated/updated PRD is automatically saved to a markdown file in the `prds/` directory for easy access, sharing, or further processing (e.g., conversion to DOCX or import to Confluence). This tool exemplifies AI-driven documentation with intelligent update capabilities.

Tools are registered in a list and bound to the LLM, enabling conditional execution via LangGraph's `tools_condition`. This approach aligns with LangChain's tool-calling paradigm, promoting modularity and extensibility.

In [22]:
# Tools
@tool
def browse_web(query: str) -> str:
    """Browse web for information using Tavily."""
    from tavily import TavilyClient
    client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))
    try:
        response = client.search(query=query)
        results = response.get("results", [])
        if results:
            return "\n".join([f"{r['title']}: {r['content']}" for r in results[:3]])
        return "No info found."
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def save_to_memory(info: str) -> str:
    """Save information to memory."""
    global memory
    memory.append(info)
    return f"Saved: {info}"

@tool
def read_file(file_path: str) -> str:
    """Read the contents of a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return f"File content from {file_path}:\n\n{content}"
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error reading file {file_path}: {str(e)}"

@tool
def edit_file(file_path: str, old_string: str, new_string: str) -> str:
    """Edit a file by replacing old_string with new_string."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        if old_string not in content:
            return f"Error: The string '{old_string}' was not found in {file_path}"
        
        new_content = content.replace(old_string, new_string, 1)
        
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        return f"Successfully edited {file_path}"
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error editing file {file_path}: {str(e)}"

@tool
def apply_patch(file_path: str, patch_content: str) -> str:
    """Apply a patch/diff to a file."""
    try:
        # Parse the patch format (similar to VS Code's applyPatch tool)
        lines = patch_content.strip().split('\n')
        if not lines:
            return "Error: Empty patch content"
        
        # Read the original file
        with open(file_path, 'r', encoding='utf-8') as f:
            original_content = f.read()
        
        # Simple patch application (for basic cases)
        # This is a simplified version - production would need more robust parsing
        updated_content = original_content
        
        for line in lines:
            if line.startswith('---') or line.startswith('+++') or line.startswith('@@'):
                continue
            elif line.startswith('-'):
                # Remove line
                line_content = line[1:].strip()
                if line_content in updated_content:
                    updated_content = updated_content.replace(line_content + '\n', '', 1)
                    updated_content = updated_content.replace(line_content, '', 1)
            elif line.startswith('+'):
                # Add line
                line_content = line[1:].strip()
                # For simplicity, append to end (real implementation would use line numbers)
                if updated_content and not updated_content.endswith('\n'):
                    updated_content += '\n'
                updated_content += line_content + '\n'
        
        # Write back the updated content
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(updated_content)
        
        return f"Successfully applied patch to {file_path}"
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error applying patch to {file_path}: {str(e)}"

@tool
def generate_prd(feature: str, description: str, user_input: str, update_existing: bool = False) -> str:
    """Generate or update PRD dynamically based on user input analysis and save to markdown file."""
    
    # Check if PRD file already exists
    import os
    safe_feature = feature.replace(" ", "_").replace("/", "_")
    filename = f"PRD_{safe_feature}.md"
    prd_dir = "prds"
    os.makedirs(prd_dir, exist_ok=True)
    filepath = os.path.join(prd_dir, filename)
    
    existing_content = ""
    if os.path.exists(filepath) and update_existing:
        # Read existing PRD content for updating
        with open(filepath, "r", encoding="utf-8") as f:
            existing_content = f.read()
    
    if update_existing and existing_content:
        prompt = f"""
You are updating an existing PRD. Here is the current PRD content:

{existing_content}

Now analyze the new user input: "{user_input}" for feature "{feature}" with description "{description}".

Update the existing PRD by incorporating the new information. Maintain the same structure but modify relevant sections based on the new input. Keep all existing good content and only update what's necessary.

Return the COMPLETE updated PRD in markdown format.
"""
    else:
        prompt = f"""
Analyze the user's input: "{user_input}" for feature "{feature}" with description "{description}".
Generate a complete PRD in markdown with the following sections, filling each based on analysis:
- Introduction (Purpose, Scope, Objectives)
- User Stories (Generate 3-5 user stories using this specific format for each:

## Description
As a [specific user type/role], I want to [specific action/goal], so that [specific benefit/value].

## Entry Point
Entry Point: [specific entry point or feature name]
Figma Link: [if applicable, otherwise omit]

## Pre-Condition
[Specific conditions that must be met before using this feature]
- User has logged in with appropriate credentials
- User has required entitlements/permissions
- User has navigated to the specific page/feature

## Done When/Acceptance Criteria
[Specific, measurable criteria for completion]
- Functional requirements (what the feature does)
- UI/UX requirements (what user sees and interacts with)
- Performance requirements
- Security requirements

## Exception Handling
[How the system handles errors or edge cases]

## General BO handling
[General back office system behaviors and standards]
)
- Functional Requirements (core features)
- Non-Functional Requirements (performance, security, etc.)
- Assumptions
- Dependencies
- Risks and Mitigations
- Timeline (realistic phases)
- Stakeholders
- Metrics

Ensure all content is generated dynamically from the analysis, no hardcoded text. Use the Everest Back Office context where appropriate.
"""
    
    response = llm.invoke(prompt)
    prd_content = response.content
    
    # Save to markdown file
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(prd_content)
    
    action = "updated" if update_existing and existing_content else "generated"
    return f"PRD {action} and saved to {filepath}\n\n{prd_content}"

# Register all tools
tools = [browse_web, save_to_memory, read_file, edit_file, apply_patch, generate_prd]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Global memory
memory: List[str] = []

## Section Explanation: State and Graph

**How it Works**: Extends LangGraph's `MessagesState` to include a custom `memory` field, enabling stateful conversations. The graph is constructed with nodes representing agent behaviors and conditional edges for dynamic routing, embodying the ReAct loop.

**What it Does**:
- **`BrainstormState` Class**: Inherits from `MessagesState` (pre-built for message handling with `add_messages` reducer) and adds a `List[str]` for memory. This supports accumulating insights across turns, crucial for iterative brainstorming.
- **Node Definitions**:
  - **`assistant`**: Core reasoning node. Invokes the LLM with system prompt and state messages, returning responses that may include tool calls. It preserves memory for continuity.
  - **`tools`**: Executes tool calls via `ToolNode`, which automatically routes to the appropriate function (e.g., `browse_web`). Results are appended to messages.
  - **`human`**: Simulates human-in-the-loop by prompting for input, appending it to state. In production, this could integrate with UI frameworks.
- **Graph Construction**:
  - **Nodes**: Added via `add_node` for modularity.
  - **Edges**: `START` to `assistant` initiates flow. Conditional edges from `assistant` use `tools_condition` to route to `tools` (if tool call detected) or `human` (otherwise). `tools` loops back to `assistant` for continued reasoning.
  - **Compilation**: `compile()` optimizes the graph for execution, enabling visualization and deployment.

This structure implements ReAct: Reason (assistant), Act (tools), Observe (results), Repeat. It leverages LangGraph's built-in reducers and conditions for robustness, as seen in official documentation.

In [23]:
# Custom State with memory
class BrainstormState(MessagesState):
    memory: List[str]

# Nodes
def assistant(state: BrainstormState):
    sys_msg = SystemMessage(content="""You are a brainstorming agent for Product Owners. Help brainstorm features, research, and discuss ideas through conversation.

IMPORTANT GUIDELINES:
- Only use generate_prd tool when user EXPLICITLY asks to "generate PRD", "create PRD", "edit PRD", "update PRD", or similar direct requests
- For general brainstorming, feature discussion, or research - just respond normally without calling tools
- If user wants to modify existing PRD, use generate_prd tool to update the existing file
- Use browse_web tool only when user asks for research or information gathering
- Use save_to_memory tool when user provides important information to remember
- Use read_file tool when user wants to examine existing files or documents
- Use edit_file tool when user wants to make specific text replacements in files
- Use apply_patch tool when user provides a patch/diff to apply to a file
- Keep conversations natural and engaging""")
    response = llm_with_tools.invoke([sys_msg] + state["messages"])
    return {"messages": [response], "memory": state["memory"]}

def human_in_loop(state: BrainstormState):
    # In interactive mode, input is handled externally
    # For simulation, can return state as-is or prompt
    # In production, integrate with UI
    return state  # Placeholder, input handled externally

# Build graph
builder = StateGraph(BrainstormState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("human", human_in_loop)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
builder.add_edge("human", "assistant")  # Allow human to loop back to assistant

graph = builder.compile()

In [24]:
# Example conversation
initial_state = {
    "messages": [HumanMessage(content="I want to add a reporting feature for Power Cash so users can see their activity more clearly and stay engaged with the app.")],
    "memory": []
}

# Run graph (simulasi)
result = graph.invoke(initial_state)
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}")

# Memory
print("Memory:", result["memory"])

human: I want to add a reporting feature for Power Cash so users can see their activity more clearly and stay engaged with the app.
ai: That sounds like a great addition to Power Cash! A reporting feature can really help users track their financial activities and make more informed decisions. Let's brainstorm some ideas for this feature:

1. **Customized Reports**: Allow users to generate reports based on date ranges, transaction types, and categories. This will provide flexibility in how they view their data.

2. **Visualizations**: Include charts and graphs to provide a visual representation of spending habits, income vs. expenses, or monthly trends. This can help users quickly grasp patterns.

3. **Daily/Weekly/Monthly Summaries**: Offer summary reports at different frequencies to help users stay updated with their financial status.

4. **Export Options**: Enable users to export reports to PDF, CSV, or Excel formats for offline analysis or sharing with financial advisors.

5. **Noti

In [30]:
# Interactive Execution with Human-in-the-Loop
def run_interactive_session(max_iterations=20):
    """Run interactive brainstorming session with safeguards against infinite loops."""
    # Initial state
    state = {
        "messages": [HumanMessage(content="I want to add a reporting feature for Power Cash so users can see their activity more clearly and stay engaged with the app.")],
        "memory": []
    }
    
    print("=== Brainstorming Session Started ===")
    print("Initial PO Input:", state["messages"][0].content)
    print(f"Tips: Press Ctrl+C to stop anytime, or type 'exit' when prompted for input.")
    print(f"PRD Generation: Only occurs if you explicitly request 'generate PRD' or 'edit PRD'")
    print(f"File Tools: Use 'read file [path]', 'edit file [path] replace [old] with [new]', or 'apply patch [path] [content]'")
    print(f"Max iterations: {max_iterations} (to prevent infinite loops)")
    print()
    
    iteration_count = 0
    last_response = ""
    
    while iteration_count < max_iterations:
        try:
            iteration_count += 1
            print(f"=== Iteration {iteration_count} ===")
            print("DEBUG: About to invoke graph...")
            print(f"DEBUG: Current state messages count: {len(state['messages'])}")
            print(f"DEBUG: Current state memory: {state['memory']}")
            
            # Invoke graph with timeout handling
            try:
                result = graph.invoke(state)
                print("DEBUG: Graph invoked successfully")
                print(f"DEBUG: Result messages count: {len(result['messages'])}")
                print(f"DEBUG: Result memory: {result['memory']}")
            except KeyboardInterrupt:
                print("\nAPI call interrupted by user (Ctrl+C).")
                print("You can try again or exit the session.")
                try:
                    user_choice = input("Try again? (y/n): ")
                    if user_choice.lower() == 'y':
                        iteration_count -= 1  # Don't count this as a full iteration
                        continue
                    else:
                        break
                except KeyboardInterrupt:
                    print("\nExiting session...")
                    break
            
            # Update state with result
            state = result
            print("DEBUG: State updated with result")
            
            # Print latest AI response
            latest_msg = result["messages"][-1]
            if hasattr(latest_msg, 'tool_calls') and latest_msg.tool_calls:
                print("DEBUG: AI response contains tool calls")
                if latest_msg.content and latest_msg.content.strip():
                    print(f"AI: {latest_msg.content}")
                else:
                    print("AI: [Making tool calls...]")
                print(f"Tool calls: {[tc['name'] for tc in latest_msg.tool_calls]}")
            else:
                print(f"AI: {latest_msg.content}")
            print("DEBUG: AI response printed")
            print()
            
            # Check for infinite loop (similar responses)
            if latest_msg.content == last_response and iteration_count > 2:
                print("Warning: Agent is giving the same response repeatedly. Might be stuck.")
                try:
                    user_input = input("Continue? (y/n): ")
                    if user_input.lower() != 'y':
                        break
                except KeyboardInterrupt:
                    print("\nExiting session...")
                    break
            
            last_response = latest_msg.content
            
            # Check if we need human input (if last message is from AI and no tool calls)
            if latest_msg.type == "ai" and (not hasattr(latest_msg, 'tool_calls') or not latest_msg.tool_calls):
                print("DEBUG: No tool calls detected, asking for human input...")
                # Ask for human feedback
                try:
                    user_input = input("PO (type 'exit' to stop, 'generate PRD' to create PRD, 'read file [path]' to examine files, 'edit file [path]' to modify files, or provide feedback): ")
                    print(f"DEBUG: User input received: '{user_input}'")
                    if user_input.lower() == 'exit':
                        print("Session ended by user.")
                        break
                    elif user_input.strip():  # Only add non-empty input
                        print("DEBUG: Adding user input to messages...")
                        # Add human message to state
                        state["messages"].append(HumanMessage(content=user_input))
                        print("DEBUG: User input added to state")
                        # Continue to next iteration for assistant to process the input
                    else:
                        print("Empty input, continuing...")
                        print("DEBUG: Empty input, state not updated")
                        # Don't update state, let it continue
                        continue  # Explicitly continue to prevent duplicate invocation
                except KeyboardInterrupt:
                    print("\nSession interrupted during input.")
                    break
            else:
                print("DEBUG: Tool calls detected, processing tools...")
                print("Processing tools...")
                # Add small delay to prevent rapid API calls
                import time
                time.sleep(1)
        
        except KeyboardInterrupt:
            print("\nSession interrupted during graph execution.")
            break
        except Exception as e:
            print(f"Error during execution: {str(e)}")
            print("DEBUG: Exception occurred, continuing session...")
            # Continue the loop to allow retry
    
    if iteration_count >= max_iterations:
        print(f"Session stopped after reaching max iterations ({max_iterations})")
    
    # Final memory
    print("Final Memory:", state["memory"] if 'state' in locals() else "No results yet")

# Uncomment to run interactive session
run_interactive_session()

=== Brainstorming Session Started ===
Initial PO Input: I want to add a reporting feature for Power Cash so users can see their activity more clearly and stay engaged with the app.
Tips: Press Ctrl+C to stop anytime, or type 'exit' when prompted for input.
PRD Generation: Only occurs if you explicitly request 'generate PRD' or 'edit PRD'
File Tools: Use 'read file [path]', 'edit file [path] replace [old] with [new]', or 'apply patch [path] [content]'
Max iterations: 20 (to prevent infinite loops)

=== Iteration 1 ===
DEBUG: About to invoke graph...
DEBUG: Current state messages count: 1
DEBUG: Current state memory: []
DEBUG: Graph invoked successfully
DEBUG: Result messages count: 2
DEBUG: Result memory: []
DEBUG: State updated with result
AI: Introducing a reporting feature to Power Cash is a great idea to enhance user engagement and provide valuable insights into their financial activities. Here are some potential features and ideas you might consider for this reporting feature:

1. 

## Section Explanation: Execution and Studio

**How it Works**: Demonstrates graph invocation with sample input, showcasing the ReAct loop in action. Integrates LangGraph Studio for interactive debugging and visualization, aligning with LangGraph's development workflow.

**What it Does**:
- **Execution Example**: Invokes the graph with a PO's initial query (e.g., "Add reporting feature to Power Cash"). The agent browses, analyzes, generates PRD, and handles human feedback in a loop. Outputs include message history and memory for traceability.
- **LangGraph Studio Integration**: Provides setup for local development server. Running `langgraph dev` in the `/studio` directory launches a web UI at `http://127.0.0.1:2024`, allowing visual graph inspection, step-by-step execution, and real-time testing. This is essential for refining agent behavior, as per LangGraph's official guides.
- **Best Practices**: Includes disclaimers for Colab incompatibility and links to documentation, ensuring users follow recommended deployment paths.

This section bridges prototyping (notebook) with production (studio), emphasizing iterative development in LangGraph ecosystems.

## LangGraph Studio

**⚠️ DISCLAIMER**

Since the filming of these videos, we've updated Studio so that it can be run locally and opened in your browser. This is now the preferred way to run Studio (rather than using the Desktop App as shown in the video). See documentation [here](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server) on the local development server and [here](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server). To start the local development server, run the following command in your terminal in the `/studio` directory in this module:

```
langgraph dev
```

You should see the following output:
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

Open your browser and navigate to the Studio UI: `https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`.
Load the `brainstorming_agent` in Studio, which uses `module-7/studio/brainstorming_agent.py` set in `module-7/studio/langgraph.json`.

In [ ]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

## How to Use
1. **Environment Setup**: Ensure API keys for OpenAI and Tavily are configured in your environment or `.env` file.
2. **Run Setup Cells**: Execute the installation and import cells to prepare dependencies.
3. **Invoke the Graph**: Run the example cell for a single execution, or use the interactive session cell for full human-in-the-loop experience.
4. **Interactive Session**: Uncomment and run `run_interactive_session()` in the interactive cell. The agent will respond, and you'll be prompted for feedback after each AI response.
5. **Brainstorming**: Have natural conversations about features, ideas, and requirements. The agent will only use tools when explicitly requested.
6. **PRD Generation**: Type "generate PRD" or "edit PRD" when you want to create or update the PRD document. The agent will intelligently update existing files rather than creating duplicates.
7. **Human-in-the-Loop**: Provide feedback or additional input when prompted. Type 'exit' to end the session.
8. **Iterate Until Satisfaction**: Continue the loop until the PO approves the output.
9. **Debug with Studio**: Use LangGraph Studio for visual tracing and adjustments.

**PRD Behavior**:
- Only generates/updates when explicitly requested
- Updates existing PRD files instead of creating duplicates
- Preserves existing content while incorporating new information
- Uses professional Everest Back Office user story templates

**File Manipulation Tools**:
- Use "read file [path]" to examine existing documents
- Use "edit file [path] replace [old_text] with [new_text]" for precise edits
- Use "apply patch [path] [patch_content]" to apply diffs or patches
- All tools work with absolute or relative file paths

## Conclusion

This notebook has demonstrated the creation of a sophisticated brainstorming agent for Product Owners using LangGraph's ReAct architecture. Key takeaways include:

- **ReAct Integration**: Combining reasoning (LLM) with acting (tools) for dynamic, iterative workflows.
- **State Management**: Using custom state classes and reducers to maintain context across interactions.
- **Tool Orchestration**: Binding multiple tools to the LLM for seamless external integrations (e.g., web search, memory, PRD generation).
- **Human-in-the-Loop**: Enabling collaborative refinement through conditional routing and user feedback.
- **Studio Deployment**: Leveraging LangGraph Studio for visualization, debugging, and production-ready deployment.

By building on foundational concepts from earlier modules (chains, routers, agents), this example showcases advanced LangGraph capabilities for real-world AI applications. Experiment with customizations to adapt the agent to your specific use cases, and explore LangGraph's ecosystem for further enhancements.

For more resources, refer to the [LangGraph Documentation](https://langchain-ai.github.io/langgraph/) and [LangChain Academy](https://academy.langchain.com/) tutorials.

In [26]:
# Test the new file manipulation tools
print("=== Testing New File Manipulation Tools ===")

# Test read_file tool
print("\n1. Testing read_file tool:")
try:
    result = read_file("README.md")
    print("✓ read_file tool works")
    print(f"Content preview: {result[:100]}...")
except Exception as e:
    print(f"✗ read_file tool failed: {e}")

# Test edit_file tool (create a test file first)
print("\n2. Testing edit_file tool:")
try:
    # Create a test file
    with open("test_file.txt", "w") as f:
        f.write("Hello World\nThis is a test file.")
    
    # Test editing it
    result = edit_file("test_file.txt", "Hello World", "Hello Universe")
    print("✓ edit_file tool works")
    print(f"Result: {result}")
    
    # Verify the change
    with open("test_file.txt", "r") as f:
        content = f.read()
    print(f"File content after edit: {content}")
    
    # Clean up
    import os
    os.remove("test_file.txt")
    
except Exception as e:
    print(f"✗ edit_file tool failed: {e}")

# Test apply_patch tool
print("\n3. Testing apply_patch tool:")
try:
    # Create a test file
    with open("patch_test.txt", "w") as f:
        f.write("Line 1\nLine 2\nLine 3")
    
    # Apply a simple patch
    patch_content = """--- a/patch_test.txt
+++ b/patch_test.txt
@@ -1,3 +1,4 @@
 Line 1
+New Line 2.5
 Line 2
 Line 3"""
    
    result = apply_patch("patch_test.txt", patch_content)
    print("✓ apply_patch tool works")
    print(f"Result: {result}")
    
    # Clean up
    os.remove("patch_test.txt")
    
except Exception as e:
    print(f"✗ apply_patch tool failed: {e}")

print("\n=== File Manipulation Tools Test Complete ===")

=== Testing New File Manipulation Tools ===

1. Testing read_file tool:
✓ read_file tool works
Content preview: File not found: README.md...

2. Testing edit_file tool:
✗ edit_file tool failed: BaseTool.__call__() takes from 2 to 3 positional arguments but 4 were given

3. Testing apply_patch tool:
✗ apply_patch tool failed: 'str' object has no attribute 'parent_run_id'

=== File Manipulation Tools Test Complete ===


/tmp/ipykernel_5221/2445858530.py:7: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = read_file("README.md")
